In [1]:
import numpy as np
import re
from tqdm import tqdm
from tensorflow.keras.models import Model,load_model,Sequential
#from tensorflow.keras.layers import *
from tensorflow.keras.layers import *
import datetime
from nltk.corpus import stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

In [2]:
srcFile='d:/Ebooks/Shekspeare_sonnets.txt'

In [3]:
fp=open(srcFile,'r',encoding='UTF-8')
data=fp.read()
fp.close()

In [4]:
ignoreList=[' ']
sw=stopwords.words('english')

In [5]:
cleanData=[re.sub(pattern='[^a-zA-Z0-9]',string=x.lower(),repl=' ') for x in data.split('.')] 
#if x not in sw]

In [6]:
tok=Tokenizer()
tok.fit_on_texts(cleanData)
seq=tok.texts_to_sequences(cleanData)
maxlen=max([len(x) for x in seq])
#padded_seq=pad_sequences(seq,maxlen=maxlen)
nb_vocab=len(list(tok.word_index.keys()))+1
maxlen,nb_vocab

(139, 3238)

In [7]:
print(seq[:1])
for s in seq[:1]:
    for i in range(0,len(s)-1):
        print(s[:i+1],s[i+1])

[[2, 1359, 1360, 36, 417, 875, 172, 219, 508, 7, 876, 43, 11, 356, 102, 161, 199, 19, 23, 2, 877, 64, 32, 44, 638, 28, 313, 639, 102, 189, 28, 276, 19, 10, 878, 3, 65, 88, 220, 55, 879, 56, 9, 314, 11, 640, 13, 33, 1361, 1362, 200, 17, 1363, 72, 509, 201, 9, 33, 9, 1364, 3, 9, 54, 33, 139, 277, 10, 7, 57, 63, 2, 81, 11, 315, 418, 1, 357, 1365, 3, 2, 1366, 419, 221, 65, 88, 880, 1367, 9, 881, 1, 313, 882, 883, 56, 316, 8, 1368, 256, 2, 81, 38, 420, 30, 1369, 22, 3, 641, 2, 81, 11, 358, 32, 2, 642, 1, 20]]
[2] 1359
[2, 1359] 1360
[2, 1359, 1360] 36
[2, 1359, 1360, 36] 417
[2, 1359, 1360, 36, 417] 875
[2, 1359, 1360, 36, 417, 875] 172
[2, 1359, 1360, 36, 417, 875, 172] 219
[2, 1359, 1360, 36, 417, 875, 172, 219] 508
[2, 1359, 1360, 36, 417, 875, 172, 219, 508] 7
[2, 1359, 1360, 36, 417, 875, 172, 219, 508, 7] 876
[2, 1359, 1360, 36, 417, 875, 172, 219, 508, 7, 876] 43
[2, 1359, 1360, 36, 417, 875, 172, 219, 508, 7, 876, 43] 11
[2, 1359, 1360, 36, 417, 875, 172, 219, 508, 7, 876, 43, 11] 3

In [8]:
in_seq=[]
out_seq=[]
#arr=['a','b','c','d','e']
for s in tqdm(seq):
    for i in range(0,len(s)-1):
        in_seq.append(s[:i+1])
        out_seq.append(s[i+1])

100%|█████████████████████████████████████████████████████████████████████████████| 397/397 [00:00<00:00, 21262.34it/s]


In [9]:
maxlen=max([len(x) for x in in_seq])
in_seq=pad_sequences(in_seq,maxlen=maxlen)

In [10]:
in_seq=np.array(in_seq)
#in_seq=in_seq.reshape((in_seq.shape[0],1,3))
in_seq.shape

(17729, 138)

In [11]:
out_seq=np.array(out_seq)
out_seq.shape

(17729,)

In [12]:
outMatrix=np.zeros((out_seq.shape[0],nb_vocab))

In [13]:
for i,w in enumerate(out_seq):
    outMatrix[i][w]=1
outMatrix.shape

(17729, 3238)

In [14]:
inp=Input(shape=(maxlen,))
x=Embedding(output_dim=256,input_dim=nb_vocab,input_length=maxlen)(inp)
x=LSTM(128,return_sequences=True,activation='sigmoid')(x)
x=LSTM(128,activation='sigmoid')(x)
#x=Dense(1000,activation='softmax')(x)
#x=Dropout(rate=0.5)(x)
x=Dense(500,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.30)(x)
x=Dense(100,activation='relu')(x)
x=BatchNormalization()(x)
x=Dropout(rate=0.20)(x)
out=Dense(nb_vocab,activation='softmax')(x)

model=Model(inp,out)
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 138)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 138, 256)          828928    
_________________________________________________________________
lstm (LSTM)                  (None, 138, 128)          197120    
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 500)               64500     
_________________________________________________________________
batch_normalization (BatchNo (None, 500)               2000      
_________________________________________________________________
dropout (Dropout)            (None, 500)               0     

In [16]:
def generateTrainBatch(batch_size):
    while True:
        x_batch=np.zeros((batch_size,in_seq.shape[1]))
        y_batch=np.zeros((batch_size,outMatrix.shape[1]))
        for i in range(batch_size):
            rIdx=np.random.randint(len(in_seq)-1)
            x_batch[i]=in_seq[rIdx]
            y_batch[i]=outMatrix[rIdx]
        yield x_batch,y_batch

In [17]:
def generateValidationBatch(batch_size):
    while True:
        x_batch=np.zeros((batch_size,in_seq.shape[1]))
        y_batch=np.zeros((batch_size,outMatrix.shape[1]))
        for i in range(batch_size):
            rIdx=np.random.randint(len(in_seq)-1)
            x_batch[i]=in_seq[rIdx]
            y_batch[i]=outMatrix[rIdx]
        yield x_batch,y_batch

In [18]:
batch_size=300
spe=len(in_seq)//batch_size
spe

59

In [19]:
hist=model.fit(generator=generateTrainBatch(batch_size),steps_per_epoch=spe,epochs=1000,
                         validation_steps=generateValidationBatch(batch_size),verbose=1)

TypeError: Unrecognized keyword arguments: {'generator': <generator object generateTrainBatch at 0x0000027F98B86DC8>}

In [ ]:
hist=model.fit(in_seq,outMatrix,epochs=1000,batch_size=200,verbose=1,validation_split=0.01)

Train on 17551 samples, validate on 178 samples
Epoch 1/1000
17551/17551 [==============================] - 72s 4ms/sample - loss: 7.7340 - acc: 0.0301 - val_loss: 7.7163 - val_acc: 0.0281
Epoch 2/1000
17551/17551 [==============================] - 62s 4ms/sample - loss: 6.4654 - acc: 0.0590 - val_loss: 7.3144 - val_acc: 0.0281
Epoch 3/1000
17551/17551 [==============================] - 63s 4ms/sample - loss: 5.8563 - acc: 0.0863 - val_loss: 8.7904 - val_acc: 0.0112
Epoch 4/1000
17551/17551 [==============================] - 61s 3ms/sample - loss: 5.4787 - acc: 0.1131 - val_loss: 8.2008 - val_acc: 0.0169
Epoch 5/1000
17551/17551 [==============================] - 59s 3ms/sample - loss: 5.1352 - acc: 0.1452 - val_loss: 7.9673 - val_acc: 0.0225
Epoch 6/1000
17551/17551 [==============================] - 60s 3ms/sample - loss: 4.8118 - acc: 0.1816 - val_loss: 9.7586 - val_acc: 0.0056
Epoch 7/1000
17551/17551 [==============================] - 60s 3ms/sample - loss: 4.4959 - acc: 0.2151 - 

In [19]:
rndIdx=np.random.randint(len(in_seq)-1)
txt=in_seq[rndIdx]
print(' '.join([tok.index_word[x] for x in txt if x>0]))
txt=txt.reshape((1,maxlen))
print(txt.shape)
pred=np.argmax(model.predict(txt))
print(tok.index_word[pred])

the deuce i was going to tell you there s one thing that worries me my burner what
(1, 158)
last


In [ ]:
savedModel=load_model('d:All_AI_Related/textgenmodel.data')

In [22]:
test="this was one of"

rndIdx=np.random.randint(len(in_seq)-1)
txt=in_seq[rndIdx]
print("Testing with:",' '.join([tok.index_word[x] for x in txt if x>0]),"\n")
out=[]
for i in range(30):
    txt=txt.reshape((1,maxlen))
    preds=model.predict(txt)
    pred_word_idx=np.argmax(preds)
    pred_word=tok.index_word[pred_word_idx]
    print('next pred word:',pred_word,' next 5 best:[',[tok.index_word[x] for x in np.argsort(preds[0])[::-1][:6][1:]],']')
    txt=np.array([x for x in txt[0][1:]]+[pred_word_idx]).reshape((1,maxlen))
    out.append(pred_word)
print('Result:',' '.join(out))

Testing with: his punctuality is well known he never arrives too soon or too late and i should not 

next pred word: go  next 5 best:[ ['last', 'reach', 'get', 'wager', 'stay'] ]
next pred word: replied  next 5 best:[ ['for', 'well', 'to', 'not', 'only'] ]
next pred word: you  next 5 best:[ ['he', 'cried', 'i', 'is', 'we'] ]
next pred word: go  next 5 best:[ ['get', 'have', 'know', 'do', 'will'] ]
next pred word: for  next 5 best:[ ['to', 'replied', 'well', 'not', 'only'] ]
next pred word: your  next 5 best:[ ['you', 'last', 'go', 'time', 'least'] ]
next pred word: servant  next 5 best:[ ['wager', 'journey', 'honour', 'master', 'passport'] ]
next pred word: cried  next 5 best:[ ['t', 'you', 'go', 'get', 'will'] ]
next pred word: cried  next 5 best:[ ['you', 'colonel', 'do', 'captain', 'stuart'] ]
next pred word: stuart  next 5 best:[ ['captain', 'colonel', 'night', 'you', 'judge'] ]
next pred word: cried  next 5 best:[ ['replied', 'is', 'you', 'not', 'only'] ]
next pred word: stuart  n

In [ ]:
model.save(filepath='d:All_AI_Related/textgenmodel.data')